# Fine Tuning TrOCR with handwritten data

In [ ]:
# Import necessary modules
from data_loader import *
from model_training import *

In [ ]:
# Load the data and split it into training, testing, and validation sets
df = load_data_paths('data.csv')
train, test, val = split_data(df, random_seed=42,test_size=0.2, val_size=0.2)


In [ ]:
# Load the processor of the model
processor = load_processor("path/to/trocr")

In [ ]:
# Create the datasets for training, testing, and evaluation using the HandWrittenDataset class 
train_dataset = HandWrittenDataset(root_dir='.',
                           df=train,
                           processor=processor)
eval_dataset = HandWrittenDataset(root_dir='.',
                           df=val,
                           processor=processor)
test_dataset = HandWrittenDataset(root_dir='.',
                            df=test,
                            processor=processor)

In [ ]:
# Create the dataloaders for training, testing, and evaluation using the batch_data_loader function
train_dataloader, eval_dataloader = batch_data_loader(train_dataset, eval_dataset, batch_size=24)

## Check data dimensions, size, and processor functionality before fine-tuning 

In [ ]:
# Check the number of training, testing, and validation examples
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))
print("Number of testing examples:", len(test_dataset))

In [ ]:
# Check the shape of the first example in the training dataset
encoding = train_dataset[0]
for k,v in encoding.items():
  print(k, v.shape)

In [ ]:
# show the first image in the training dataset
image = Image.open(train['img_path'][0]).convert("RGB")
image

In [ ]:
# use the pre-trained model to predict the labels of the first example in the training dataset
labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print(label_str)

## Model Training

In [ ]:
# Set the device to be used for training and load the model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_model("path/to/trocr")

In [ ]:
# Set the model parameters of the model
model = set_model_params(model, processor, max_length=64,early_stopping=True,no_repeat_ngram_size=3,length_penalty=2.0,num_beams=4)

In [ ]:
# Set the optimizer and train the model using the train_model function
optimizer = AdamW(model.parameters(), lr=5e-5)
model = train_model(model, train_dataloader, eval_dataloader, optimizer, device)


In [ ]:
# Save the fine-tuned model
model = save_model(model, "path/to/fine-tuned-trocr")